## Name: Andrew Caide
### CSCI S-89C Deep Reinforcement Learning  
### Part II of Assignment 4

## Problem 1 (10 points)

Consider Environment that has five states: 1, 2, 3, 4, and 5. Possible transitions are: (1) 1->1, 1->2; (2) 2->1, 2->2, 2->3; (3) 3->2, 3->3, 3->4; (4) 4->3, 4->4, 4->5; (5) 5->4, 5->5.

Actions of the Agent are decoded by -1, 0, and +1, which correspond to its intention to move left, stay, and move right, respectively. The Environment, however, does not always respond to these intentions exactly, and there is 10% chance that action 0 will result in moving to the left (if moving to the left is admissible), and +1 action will result in staying - in other words, there is an "east wind." More specifically, the non-zero transition probabilities $p(s^\prime,r|s,a)$ are<br>

$p(s^\prime=1,r=0|s=1,a=0)=1$,<br>
$p(s^\prime=1,r=0|s=1,a=+1)=0.1,p(s^\prime=2,r=0|s=1,a=+1)=0.9$,<br>

$p(s^\prime=1,r=0|s=2,a=-1)=1$,<br>
$p(s^\prime=1,r=0|s=2,a=0)=0.1,p(s^\prime=2,r=0|s=2,a=0)=0.9$,<br>
$p(s^\prime=2,r=0|s=2,a=+1)=0.1,p(s^\prime=3,r=1|s=2,a=+1)=0.9$,<br>

$p(s^\prime=2,r=0|s=3,a=-1)=1$,<br>
$p(s^\prime=2,r=0|s=3,a=0)=0.1,p(s^\prime=3,r=1|s=3,a=0)=0.9$,<br>
$p(s^\prime=3,r=1|s=3,a=+1)=0.1,p(s^\prime=4,r=0|s=3,a=+1)=0.9$,<br>

etc.

Further, we assume that whenever the process enters state 3, the Environment generates reward = 1. In all other cases the reward is 0. For example, transition 2->3 will result in reward 1, transition 3->3 will result in reward 1, transition 3->2 will result in reward 0, transition 2->2 will result in reward 0, etc.



Further, assume that the agent does not know about the wind or what rewards to expect. It chooses to stay in all states, i.e. the policy is
$\pi(-1|1)=0, \pi(0|1)=1, \pi(+1|1)=0$,<br>
$\pi(-1|2)=0, \pi(0|2)=1, \pi(+1|2)=0$,<br>
$\pi(-1|3)=0, \pi(0|3)=1, \pi(+1|3)=0$,<br>
$\pi(-1|4)=0, \pi(0|4)=1, \pi(+1|4)=0$,<br>
etc.

Please estimate the action-value function using First-Visit Monte Carlo (MC) prediction. Make sure each pair $(S_0,A_0)$ has non-zero probability of being selected. Let’s use $\gamma=0.9$ and run the episodes for $T=100$. Make some reasonable selection of tolerance $\theta$.

Based on the estimates of $q_\pi(s,a)$, please suggest a better policy. Is it optimal?

---
Setting up Agent:

In [1]:
import random
import numpy as np

class Environment:
    def __init__(self, S0 = 1):
        self.time = 0
        self.state = S0

    def admissible_actions(self):
        A = list((-1,0,1))
        if self.state == 1: A.remove(-1)
        if self.state == 5: A.remove(1) 
        return A
    
    def check_state(self):
        return self.state
    
    def set_state(self, new_state):
        self.state = new_state

    def get_reward(self, action):
        self.time += 1
        move = action
        # If (we're not in s1 and moving to the right) or (if we're on ther right and move == 1)
        if (self.state > 1 and move > -1) or (self.state == 1 and move > 0):
            move = np.random.choice([move-1, move],p=[0.1,0.9])
        self.state += move
        
        if self.state == 3:
            reward = 1
        else:
            reward = 0
        return reward
    
    
#########
    
class Agent:
    def __init__(self):
        self.current_reward = 0.0
        self.scan = 1
        self.scan_done = False

    def step(self, env, take_action = False, policy = 1):
        
        # If taking action: policy 1 = random movement, policy 2 = converge to S3
        if take_action:
            action_selected = random.choice(env.admissible_actions())
        else:
            action_selected = 0 # Stay put
        
        reward = env.get_reward(action_selected)            
        self.current_reward = reward

Please estimate the action-value function using First-Visit Monte Carlo (MC) prediction. Make sure each pair  (𝑆0,𝐴0)  has non-zero probability of being selected. 
1. Let’s use  𝛾=0.9.
2. Run the episodes for  𝑇=100. 
3. Make some reasonable selection of tolerance  𝜃.

In [2]:
def first_visit_trial(theta=0.1, State=1, gamma=0.9, act = False, policy = 1):
    
    # Initial state chosen randomly
    S0 = random.randint(1,5)
    
    # Set up Env and conditions
    env = Environment(S0)
    agent = Agent()
    G = 0
    G_old = 0
    delta = 1
    record_rewards = False
    i = 0
    
    while delta > theta:
        
        agent.step(env, act, policy)
        if env.check_state() == State:  # Once we hit St = t, we start recording rewards 
            record_rewards = True
            
        if record_rewards:
            G += G*gamma**(env.time-1) + agent.current_reward
            if i == 0:
                G_old = G
                i += 1
            else:
                delta = G - G_old
                G_old = G
                i += 1

        # We need to perform a series of checks if we expect the agent to stand still
        if not act:
            # If the current state is less than 3 and we have not hit the reward (we won't!) return 0
            if env.check_state() < 3 and not record_rewards:
                return G

            # If we under-shoot our starting-point S0, return NA - we can't record rewards
            if State > S0 and not record_rewards:
                return np.nan
    return G

In [3]:
def fv_action_estimator(theta, State, gamma, episodes = 100, act = False, policy = 1):
    S = []
    for i in range(episodes):
        S.append(first_visit_trial(theta,State,gamma, act, policy))
    return round(np.nanmean(S),3) # We're returning nan's!

In [4]:
T = 100
V = np.array([fv_action_estimator(theta=.001, State = s, gamma=0.9, episodes=100) for s in range(1,6)])
print(["V(St={}) = {}".format(i+1, v) for i, v in enumerate(V)])

['V(St=1) = 0.0', 'V(St=2) = 0.0', 'V(St=3) = 864.07', 'V(St=4) = 78.999', 'V(St=5) = 0.0']


---
Based on the estimates of  𝑞𝜋(𝑠,𝑎) , please suggest a better policy. Is it optimal?

Testing two policies: 
1. Equal chance to do any of the admissible actions
2. Always converge to $S_{3}$

In [5]:
# 1 - Random actions:
T = 100
V = np.array([fv_action_estimator(theta=.001, State = s, gamma=0.9, episodes=100, act = True) for s in range(1,6)])
print(["V(St={}) = {}".format(i+1, v) for i, v in enumerate(V)])

['V(St=1) = 0.0', 'V(St=2) = 134.01', 'V(St=3) = 872.777', 'V(St=4) = 151.546', 'V(St=5) = 0.0']


**Observation:** Policy 2 (listed above) doesn't work; First Visit MC requires the agent to touch the state we're measuring the state-value for before we start adding rewards. In the case of always-converge, there is a strong chance, when randomly placed in the environment, that the agent will **miss** the target state. 

Example: Target = 2 & S0 = 4, or Target = 1, S0 = 2. In these cases, the agent will converge to S=3 and stay there, never hitting the target (and we start infinitely looping because of our exit condition - the delta - is never satisfied). 

**Is it optimal**?

Definitely not, but I think it's *better*, by randomly acting around, the agent has a chance to explore the environment a little better. Under this situation, $V_{St}(2 and 4)$ see some rewards. Previously, $V_{St}(4)$ was seeing some rewards but it was purely due to drift. In the random walk condition, it sees more rewards but as does $V_{St}(2)$, and this can be useful in a situation where the rewards weren't solely centralized in $V_{St}(3)$ 

At the end of the simulations, $V_{St}(3)$ nets the most rewards by almost 4x, indicating that it's obviously the optimal state.

## Problem 2 (10 points)

In this problem, the agent will obtain the optimal policy via MC control. Please run the On-policy first-visit MC control (for $\varepsilon$-soft policies)  without Exploring Starts algorithm for estimating $\pi_*$. Use same $\gamma=0.9$ and $T=100$.

Does the final policy appear to be optimal?

In [6]:
# Updating agent to use a policy
import pandas as pd

class Policy_Agent:
    def __init__(self):
        self.current_reward = 0.0
        self.scan = 1
        self.scan_done = False
        self.action_selected = 0
        
        ## Help! There has to be a smarter way of doing this. 
        self.policy = {1:[0, 1/2, 1/2], 2: [1/3,1/3,1/3],
                      3: [1/3,1/3,1/3], 4: [1/3,1/3,1/3],
                      5: [1/2,1/2,0]}

    def update_policy(self, new_policy):
        self.policy = new_policy

    def step(self, env):
        
        self.action_selected = np.random.choice([-1,0,1], 1, p=self.policy[env.check_state()])[0]
        reward = env.get_reward(self.action_selected)  
        self.current_reward = reward

In [7]:
### Completely recycled; updating to kleep track of R(S, A)

# This implementation is questionable. 

def first_visit_trial(theta=0.1, State=1, gamma=0.9, new_policy = False):
    
    # Initial state chosen randomly
    S0 = random.randint(1,5)
    
    # Set up Env and conditions
    env = Environment(S0)
    agent = Policy_Agent()
    if new_policy:
        agent.update_policy(new_policy) 
    R = 0
    R_old = 0
    delta = 10
    record_rewards = False
    i = 0
    rewards = {1:{-1:0,0:0,1:0}, 2:{-1:0,0:0,1:0}, 3:{-1:0,0:0,1:0},
               4:{-1:0,0:0,1:0}, 5:{-1:0,0:0,1:0}}
    
    while delta > theta:
        
        old_state = env.check_state()
        agent.step(env)
        action = agent.action_selected
        
        if env.check_state() == State:  # Once we hit St = t, we start recording rewards 
            record_rewards = True
            
        if record_rewards:
            R += R*gamma**(env.time-1) + agent.current_reward
            if i == 0:
                R_old = R
                i += 1
            else:
                delta = R - R_old
                R_old = R
                i += 1
                rewards[old_state][action] = round(R,4)
        
    return rewards

In [8]:
# We shall recycle first_visit_trial function; it still can serve a purpose
# Need to rework the fv_action_estimator; we want a policy estimator!

def FVMC_policy_estimator(theta=1, State=3, gamma=.9, episodes = 100, eps = .3, new_policy = False):
    # There REALLY has to be a smarter way of doing this...
    
    s1 = pd.DataFrame()
    s2 = pd.DataFrame()
    s3 = pd.DataFrame()
    s4 = pd.DataFrame()
    s5 = pd.DataFrame()
    
    for i in range(episodes):
        if new_policy:
            results = first_visit_trial(theta,State,gamma, new_policy)
        else:
            results = first_visit_trial(theta,State,gamma)
        
        s1 = s1.append(results[1], ignore_index = True)
        s2 = s2.append(results[2], ignore_index = True)
        s3 = s3.append(results[3], ignore_index = True)
        s4 = s4.append(results[4], ignore_index = True)
        s5 = s5.append(results[5], ignore_index = True)
    
    results = pd.DataFrame({1: s1.mean(), 2: s2.mean(), 3:s3.mean(), 4:s4.mean(),5:s5.mean()})
    records = results.copy() #for closer examination
    
    # Get new policy
    for col in results.columns:
        A = results[col] == results[col].max()
        if col in [2, 3, 4]:
            results.loc[A,col] = 1-eps+eps/3
            results.loc[-A,col]= eps/3
        else:
            not0 = (results[col] != 0) & (-A)
            results.loc[A,col] = 1-eps+eps/2
            results.loc[not0,col] = eps/2
    return ([records, results])


In [9]:
r = FVMC_policy_estimator(State=3)
print("-------------------\nReward values:")
print(r[0])
print("\nUpdated policies:")
print(r[1])

-------------------
Reward values:
             1           2           3           4           5
-1    0.000000  400.224339  409.593904  380.926835  280.243363
 0  403.544919  391.399529  361.401660  262.717017  228.231358
 1  405.140990  413.127591  374.290122  302.952976    0.000000

Updated policies:
       1    2    3    4     5
-1  0.00  0.1  0.8  0.8  0.85
 0  0.15  0.1  0.1  0.1  0.15
 1  0.85  0.8  0.1  0.1  0.00


The results are *almost* optimal; The agent clearly wants to move to position 3, but depending on RNG (not sure what's going on here), at state 3 it will want to move either left or right, but immediately return back to 3. 

In [12]:
# Bonus, update agent with new policy and try again.

new_policy = {}
for_updating = r[1].to_dict().items()
for key, value in for_updating:
    new_policy[key] = list(value.values())

In [11]:
r = FVMC_policy_estimator(State=3, new_policy = new_policy)
print("-------------------\nReward values:")
print(r[0])
print("\nUpdated policies:")
print(r[1])

-------------------
Reward values:
             1           2           3           4          5
-1    0.000000  126.518380  139.842284  132.209775  44.417125
 0   45.601373  126.695266  118.962205   44.028881   0.000000
 1  127.900760  138.326346  130.958482   44.066135   0.000000

Updated policies:
       1    2    3    4     5
-1  0.00  0.1  0.8  0.8  0.85
 0  0.15  0.1  0.1  0.1  0.00
 1  0.85  0.8  0.1  0.1  0.00


Can't say for sure it's an improvement, but it's definitely converging faster around 3!


----